In [3]:
!pip install pymysql # mysql database connection library
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
!python3 -m pip install --upgrade "git+https://github.com/nficano/pytube.git"   # pytube-12.0.0 설치
! pip install pydub

import os
import time                     # 딜레이 주기 위함
import datetime
import numpy as np
import pandas as pd             # Querying results to make dataframe(to divide datas)
import pymysql.cursors          # For Request Query String
from google.colab import drive

import pytube
from pytube import Search
from pydub import AudioSegment

drive.mount('/content/drive', force_remount=True)

     |████████████████████████████████| 43 kB 1.2 MB/s 
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-notebook", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/jupyter_core/application.py", line 264, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/notebook/notebookapp.py", line 1507, in initialize
    self.init_webapp()
  File "/usr/local/lib/python3.7/dist-packages/notebook/notebookapp.py", line 1297, in init_webapp
    self.http_server.listen(port, self.ip)
  File "/usr/local/lib/python3.7/dist-packages/tornado/tcpserver.py", line 143, in listen
    sockets = bind_soc

In [36]:
conn = pymysql.connect(host='csrd.cksngv0eixsu.ap-northeast-2.rds.amazonaws.com', port=3306, user='admin', password='qwerty1!', \
db='before-pp', charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)        # mysql connect inform

In [37]:
sql = "SELECT c.sg_name, c.sg_artists FROM bf_pl a join bf_tg b on a.pl_id = b.pl_id join bf_sg c on c.pl_id = b.pl_id WHERE pl_intro LIKE '팝%'"
cur = conn.cursor() # open to query
cur.execute(sql)    # excute query
result = cur.fetchall() # merging result
conn.close()    # DB disconnect

In [38]:
result

[{'sg_artists': 'Ariana Grande, Justin Bieber', 'sg_name': 'Stuck with U'},
 {'sg_artists': 'Ariana Grande',
  'sg_name': "break up with your girlfriend, i'm bored (Explicit Ver.)"},
 {'sg_artists': 'Ariana Grande', 'sg_name': 'The Way (Feat. Mac Miller)'},
 {'sg_artists': 'Maroon 5', 'sg_name': "Can't Stop"},
 {'sg_artists': 'Maroon 5', 'sg_name': 'Through With You'},
 {'sg_artists': 'Maroon 5', 'sg_name': 'Sweetest Goodbye'},
 {'sg_artists': 'Little Mix', 'sg_name': 'Break Up Song'},
 {'sg_artists': 'Kehlani', 'sg_name': 'Toxic'},
 {'sg_artists': 'Tinashe', 'sg_name': '2 On (Feat. Schoolboy Q)'},
 {'sg_artists': 'Dan + Shay', 'sg_name': 'All to Myself'},
 {'sg_artists': 'Kelsea Ballerini', 'sg_name': 'Miss Me More'},
 {'sg_artists': 'Years & Years', 'sg_name': 'King'},
 {'sg_artists': 'Years & Years', 'sg_name': 'Desire'},
 {'sg_artists': 'Years & Years', 'sg_name': 'Shine'},
 {'sg_artists': 'Years & Years',
  'sg_name': "Meteorite (영화 '브리짓 존스의 베이비' 삽입곡)"},
 {'sg_artists': 'Gryffin, 

In [ ]:
df = pd.DataFrame(result)   # 전체 음악리스트 확인(playlist_id 기준이기 때문에 중복값 들어있음)
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)  # 중복 처리
ext_lists = df.values.tolist() # pytube에서 검색어로 사용하기 위한 리스트 변환

In [ ]:
genre_lists = ['락', '클래식', '메탈', '디스코', '블루스', '래게', '트로트', '힙합', '재즈', '팝']
result_merge = []
#================================================장르별 검색 루프===============================================================================

for q_param in genre_lists:
    conn = pymysql.connect(host='csrd.cksngv0eixsu.ap-northeast-2.rds.amazonaws.com', port=3306, user='admin', password='qwerty1!', \
    db='before-pp', charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)        # mysql connect inform
    sql = f"SELECT c.sg_name, c.sg_artists FROM bf_pl a join bf_tg b on a.pl_id = b.pl_id join bf_sg c on c.pl_id = b.pl_id WHERE pl_intro LIKE '{q_param}%'"
    print(sql)
    cur = conn.cursor() # open to query
    cur.execute(sql)    # excute query
    result = cur.fetchall() # merging result
    result_merge.append(result)
    conn.close()    # DB disconnect\
result_merge

,sg_name,sg_artists
0,Stuck with U,"Ariana Grande, Justin Bieber"
1,"break up with your girlfriend, i'm bored (Expl...",Ariana Grande
2,The Way (Feat. Mac Miller),Ariana Grande
3,Can't Stop,Maroon 5
4,Through With You,Maroon 5
...,...,...
438,하얘졌어,Jimmy Brown
439,Slow,SOLE (쏠)
440,Alone,JUNNY (주니)
441,No One (Feat. Minshik),Jimmy Brown


In [5]:
genre_lists = ['락', '클래식', '메탈', '디스코', '블루스', '래게', '트로트', '힙합', '재즈', '팝']

#================================================장르별 검색 루프===============================================================================

for q_param in genre_lists:
    sql = f"SELECT c.sg_name, c.sg_artists FROM bf_pl a join bf_tg b on a.pl_id = b.pl_id join bf_sg c on c.pl_id = b.pl_id WHERE pl_intro LIKE '{q_param}%'"
    cur = conn.cursor() # open to query
    cur.execute(sql)    # excute query
    result = cur.fetchall() # merging result
    conn.close()    # DB disconnect
    df = pd.DataFrame(result)   # 전체 음악리스트 확인(playlist_id 기준이기 때문에 중복값 들어있음)
    df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)  # 중복 처리
    ext_lists = df.values.tolist() # pytube에서 검색어로 사용하기 위한 리스트 변환

#================================================음원 검색===============================================================================

    audio_lists = []    # 검색결과가 나온 비디오 넣는 리스트
    forbidden_lists = []    # 검색결과 없는 비디오 리스트 - 추후 DB 적용하기 위하여
    for ext in ext_lists:
        try:
            StrClean = "-".join(ext)    # 노래 제목과 가수를 "-"를 구분자로 문자열 합치기
            yt = Search(StrClean).results[0]    # Search 메소드를 사용하여 검색하고, 그 결과의 1번째 결과 사용(빈도 상, 1 ~ 2번째 영상에 가장 관련있는 영상 표시)
        except:
            print("=================== Error Occured :  " + StrClean + " ===================")  # 에러 로그
            forbidden_lists.append(StrClean)    # 검색결과 없는 비디오 리스트 삽입
        audio_lists.append({StrClean : yt})     # 성공적으로 검색되는 비디오의 Youtube 객체 리스트 삽입
    # 추후 파일명으로 사용하여 라벨링 및 DB 적용을 수월하게 하기 위하여 딕셔너리 형태로 전달

#================================================음원 다운로드===============================================================================

    unfounded_lists = []    # 검색된 영상 중 필터링된, 관련성 떨어지는 목록용 리스트
def youtube_download(var):
    keys = str(list(var.items())[0][0])   # 딕셔너리에서 파일 이름으로 쓸 string 언패킹
    link = var.get(keys)                     # 딕셔너리에서 파일이름이 key값이므로, values 값인  (pytube.__main__.YouTube) 자료형 언패킹
    audio_streams = link.streams.filter(only_audio=True)    # pytube에서 지원하는 다양한 stream 중 Audio 속성을 가진 stream만
    # print(audio_streams)  # audio streams 처리 전체 정보

    flt1a = (int(link.length)) > 130    # 영상 길이 2분 10초 초과
    if q_param == "클래식":
        flt1b = (int(link.length)) < 1200    # 그래도 클래식이니까 영상 길이 20분 미만
    else:
        flt1b = (int(link.length)) < 600    # 클래식 제외 10분 미만
    # flt1b = (int(link.length)) < 150    # 일부러 에러를 내기 위한 조건

    flt1 = flt1a and flt1b              # 위의 두 조건 동시에 만족하는 필터
    flt2 = ("official" in str(link.title).lower() or "audio" in str(link.title).lower() or "lylic" in str(link.title).lower())  
    # "official", "audio", "lylic" 중 제목에 하나라도 들어가는 영상 제목 통과

    # 내용 체크
    print(f'영상제목: {link.title},  영상 조회수: {link.views},  영상 길이: {link.length} sec. [{str(link.length // 60).zfill(2)}:{str(link.length % 60).zfill(2)}]')
    print(f'영상 URL: {link.watch_url}')
    print(f"# 1번 체크포인트 - 음원 길이(초):  {int(link.length)}, 타입은? : {type(int(link.length))}")
    print(f"# 2번 체크포인트 - 음원 제목(소문자):  {str(link.title).lower()}") 
    print(f"# 3번 체크포인트 - 2분 이상 20분 이하 길이:  {flt1}  ,  음원에 자주 붙은 문자열 포함 여부: {flt2}")


    # 저장 폴더 만들기
    path = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/download/"
    genre_dir = q_param
    dir = path + genre_dir 
    if not os.path.exists('dir'):
        os.mkdir('dir')     

    ## filter()를 활용하여 파일 확장자 타입 등을 포함한 streams 처리
    if flt1 or flt2 == True:    # 위의 1, 2 조건 중 한가지라도 만족한다면 다운로드
        audio_streams = link.streams.filter(file_extension='mp4').get_by_itag(140)  # 여러개의 다운로드 스트림 선택해서 링크 하나 가져오기
        title = keys  # 영상 제목으로 파일명이 하고 싶다면? audio_streams.title 또는 link.title 사용 
        audio_streams.download(filename= title + '.mp3', output_path= dir)     # 설정한 파일명, 경로에 다운로드
        time.sleep(5)                                                          # 5초 휴지
        # print(f"=============== {keys.spilt('-')[1]}란 가수의 {keys.spilt('-')[0]} 란 노래 음원 다운로드 완료! ===============")    # 성공 로그
        print(f"#@#@#@#@#@#@#@ {keys} 다운로드 성공!")             # 성공 로그
        print("============================================== 구분선 ==============================================")
    else:
        print(f"^^^^^^^^^^^ Related Video Not Found : {keys}")  # 실패 로그
        unfounded_lists.append(keys)    # 실패 목록 리스트에 저장
        print("============================================== 구분선 ==============================================")
        

Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Killing In the Name-Rage Against The Machine
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Raining Blood-Slayer
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.


InterfaceError: ignored

In [ ]:
fail_cnt = 0
for fail_cnt ,audio in enumerate(audio_lists):
    try:
        youtube_download(audio)
    except:
        print(f"no error keep going, {fail_cnt}개째 다운로드 실패")

영상제목: Seong Jin Cho – Mozart:  Allegro in D major, K 626b/16 (Live from Salzburg, 2021),  영상 조회수: 50732,  영상 길이: 112 sec. [01:52]
영상 URL: https://youtube.com/watch?v=IUq1hc9TYzU
# 1번 체크포인트 - 음원 길이(초):  112, 타입은? : <class 'int'>
# 2번 체크포인트 - 음원 제목(소문자):  seong jin cho – mozart:  allegro in d major, k 626b/16 (live from salzburg, 2021)
# 3번 체크포인트 - 2분 이상 10분 이하 길이:  False  ,  음원에 자주 붙은 문자열 포함 여부: False
^^^^^^^^^^^ Related Video Not Found : Mozart : Allegro In D Major K.626b/16-조성진
============================================== 구분선 ==============================================
영상제목: Víkingur Ólafsson – Bach: Organ Sonata No. 4, BWV 528: II. Andante [Adagio] (Transcr. Stradal),  영상 조회수: 1848437,  영상 길이: 328 sec. [05:28]
영상 URL: https://youtube.com/watch?v=h3-rNMhIyuQ
# 1번 체크포인트 - 음원 길이(초):  328, 타입은? : <class 'int'>
# 2번 체크포인트 - 음원 제목(소문자):  víkingur ólafsson – bach: organ sonata no. 4, bwv 528: ii. andante [adagio] (transcr. stradal)
# 3번 체크포인트 - 2분 이상 10분 이하 길이:  True  ,  음원에 자주 붙은 문자열 

In [ ]:
def unfounded_df(ufs):   # unfounded_lists를 DB에 적용하기 위한 Dataframe 만들기
    df = pd.DataFrame(ufs, columns = ['col'])
    df['sg_name'] = df.col.str.split('-').str[0]
    df['sg_artists'] = df.col.str.split('-').str[1]
    df.drop(['col'], axis=1, inplace=True)
    return df

In [ ]:
failed_df = unfounded_df(unfounded_lists)   # 데이터프레임 확인
failed_df

,sg_name,sg_artists
0,Mozart : Allegro In D Major K.626b/16,조성진
1,"멘델스존: 무언가, 작품번호 62: 6. Allegretto grazioso (no...","Mischa Maisky, Sergio Tiempo"
2,"슈만: 5개의 노래, 작품번호 96: 3. 그의 목소리",Dietrich Fischer
3,리스트: 피아노 소나타 나단조,Krystian Zimerman
4,바흐: 크리스마스 오라토리오 / 1부: 2번 그것은 시간을 떠나 왔다네,"Anthony Rolfe Johnson, English Baroque Soloist..."
5,"드보르자크: 교향곡 9번 마단조 작품번호 95, B. 178 '신세계로부터': 4....","Rafael Kubelik, Berliner Philharmoniker"
6,Bruckner : Symphony No.8 In C Minor WAB.108,"IV. Finale. Feierlich, Nicht Schnell (브루크너 : ..."
7,Schumann : Dichterliebe Op.48,I. Im Wunderschonen Monat Mai (슈만 : 시인의 사랑 작품...
8,"모차르트: 교향곡 40번 사단조, K.550: 1. Molto allegro","Herbert Von Karajan, Berliner Philharmoniker"
9,"Prokofiev: 프로코피예프: 세 개의 오렌지에 대한 사랑, 작품번호 33",행진곡


In [ ]:
ext_lists2 = failed_df.values.tolist() # pytube에서 검색어로 사용하기 위한 리스트 변환

In [ ]:
audio_lists2 = []    # 검색결과가 나온 비디오 넣는 리스트
forbidden_lists2 = []    # 검색결과 없는 비디오 리스트 - 추후 DB 적용하기 위하여

for ext in ext_lists2:
    try:
        StrClean = "-".join(ext)    # 노래 제목과 가수를 "-"를 구분자로 문자열 합치기
        yt = Search(StrClean).results[1]    # Search 메소드를 사용하여 검색하고, 1번째 영상에서 에러가 발생했으므로 2번째 영상 선택
    except:
        print("=================== Error Occured :  " + StrClean + " ===================")  # 에러 로그
        forbidden_lists2.append(StrClean)    # 검색결과 없는 비디오 리스트 삽입
    audio_lists2.append({StrClean : yt})     # 성공적으로 검색되는 비디오의 Youtube 객체 리스트 삽입
    # 추후 파일명으로 사용하여 라벨링 및 DB 적용을 수월하게 하기 위하여 딕셔너리 형태로 전달

In [ ]:
forbidden_lists2

[]

In [ ]:
fail_cnt = 0
for fail_cnt ,audio in enumerate(audio_lists2):
    try:
        youtube_download(audio)
    except:
        print(f"no error keep going, {fail_cnt}개째 다운로드 실패")

영상제목: Mozart: Allegro in D Major, K. 626b/16,  영상 조회수: 106175,  영상 길이: 101 sec. [01:41]
영상 URL: https://youtube.com/watch?v=gHifLVl8yQs
# 1번 체크포인트 - 음원 길이(초):  101, 타입은? : <class 'int'>
# 2번 체크포인트 - 음원 제목(소문자):  mozart: allegro in d major, k. 626b/16
# 3번 체크포인트 - 2분 이상 10분 이하 길이:  False  ,  음원에 자주 붙은 문자열 포함 여부: False
^^^^^^^^^^^ Related Video Not Found : Mozart : Allegro In D Major K.626b/16-조성진
============================================== 구분선 ==============================================
영상제목: Felix Mendelssohn-Songs Without Words Op.62-6 `Fruhlingslied'-allegretto grazioso,  영상 조회수: 185,  영상 길이: 209 sec. [03:29]
영상 URL: https://youtube.com/watch?v=yIEpzOyDkBE
# 1번 체크포인트 - 음원 길이(초):  209, 타입은? : <class 'int'>
# 2번 체크포인트 - 음원 제목(소문자):  felix mendelssohn-songs without words op.62-6 `fruhlingslied'-allegretto grazioso
# 3번 체크포인트 - 2분 이상 10분 이하 길이:  True  ,  음원에 자주 붙은 문자열 포함 여부: False
#@#@#@#@#@#@#@ 멘델스존: 무언가, 작품번호 62: 6. Allegretto grazioso (no.30)-Mischa Maisky, Sergio Tiempo 다운로드 성공

In [ ]:
path = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/download/"
genre_dir = "classic"
dir  = path + genre_dir
def song_names(dir):
    files = list(os.listdir(dir))
    return files    # filename

filename = song_names(dir)
len(filename)

166

In [ ]:
def load_song(filename):
    song = f'{filename}'
    path = f'{dir}/{filename}'
    return song, path
songs = []
paths = []

for file in filename:
    song, path = load_song(file)
    songs.append(f"{song}")
    paths.append(f"{path}")

In [ ]:
ths_lists = []

for file in paths:
    filename, file_extension = os.path.splitext(file)
    if file_extension == ".mp3":
        dst = file.replace(".mp3","")+".wav"                                                             
        audSeg = AudioSegment.from_file(file, format="mp4")  # convert wav from mp3 
        audSeg.export(dst, format="wav")
        song = AudioSegment.from_wav(dst)
        sec = 10000     # 10초 변수
        block = song[sec * 8 : sec * 11]     # 1분 20초 ~ 1분 50초 구간만 사용하기 위한 변수(하이라이트 추출)
        down_dir = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/download/"
        inner_dir = f"converted/" # 테스트 버전 2
        dir = f"{down_dir}{genre_dir}/{inner_dir}"    
        if not os.path.exists(dir):
            os.mkdir(dir)
        name = f'{file.replace(".mp3","").replace(f"{down_dir}{genre_dir}/","")}_30.wav'
        export_path = dir + name
        block.export(export_path, format='wav', parameters=["-q:a", "10", "-ac", "1"])
        print(f"Converting Successful // converted filename: {export_path}")
        ths_lists.append(export_path)
    else:
        print(file)

In [ ]:
ths_lists